## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

Применим полученные навыки и решим задачу анализа тональности отзывов.

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Возможные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать гиперпараметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор.

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

from nltk.stem import PorterStemmer

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
sms_spam = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/sms_spam.csv')
sms_spam

,type,text
0,ham,Hope you are having a good week. Just checking in
1,ham,K..give back my thanks.
2,ham,Am also doing in cbe only. But have to pay.
3,spam,"complimentary 4 STAR Ibiza Holiday or £10,000 ..."
4,spam,okmail: Dear Dave this is your final notice to...
...,...,...
5554,ham,You are a great role model. You are giving so ...
5555,ham,"Awesome, I remember the last time we got someb..."
5556,spam,"If you don't, your prize will go to another cu..."
5557,spam,"SMS. ac JSco: Energy is high, but u may not kn..."


In [ ]:
ps = PorterStemmer()

def clean(text):
  processed_sentence = [ps.stem(word).lower() for word in word_tokenize(text)]
  processed_sentence = ' '.join(word for word in processed_sentence)
  return processed_sentence

sms_spam.text = sms_spam.text.apply(clean)

In [ ]:
sms_spam

,type,text
0,ham,hope you are have a good week . just check in
1,ham,k .. give back my thank .
2,ham,am also do in cbe onli . but have to pay .
3,spam,"complimentari 4 star ibiza holiday or £10,000 ..."
4,spam,okmail : dear dave thi is your final notic to ...
...,...,...
5554,ham,you are a great role model . you are give so m...
5555,ham,"awesom , i rememb the last time we got somebod..."
5556,spam,"if you do n't , your prize will go to anoth cu..."
5557,spam,"sm . ac jsco : energi is high , but u may not ..."


In [ ]:
noise = stopwords.words('english')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sms_spam.text, sms_spam.type, train_size = 0.7)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

In [ ]:
pipeline = make_pipeline(
    CountVectorizer(stop_words = noise),
    MultinomialNB()
)

parameters = {
    'countvectorizer__max_features': (None, 500, 1000, 10000),
    'countvectorizer__ngram_range': ((1, 2), (1, 3), (1, 4), (1, 5)),
}

grid_search = GridSearchCV(pipeline, parameters)

grid_search.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('countvectorizer',
                                        CountVectorizer(stop_words=['i', 'me',
                                                                    'my',
                                                                    'myself',
                                                                    'we', 'our',
                                                                    'ours',
                                                                    'ourselves',
                                                                    'you',
                                                                    "you're",
                                                                    "you've",
                                                                    "you'll",
                                                                    "you'd",
                                                                    'your',
                                                                    'yours',
                                                                    'yourself',
                                                                    'yourselves',
                                                                    'he', 'him',
                                                                    'his',
                                                                    'himself',
                                                                    'she',
                                                                    "she's",
                                                                    'her',
                                                                    'hers',
                                                                    'herself',
                                                                    'it',
                                                                    "it's",
                                                                    'its',
                                                                    'itself', ...])),
                                       ('multinomialnb', MultinomialNB())]),
             param_grid={'countvectorizer__max_features': (None, 500, 1000,
                                                           10000),
                         'countvectorizer__ngram_range': ((1, 2), (1, 3),
                                                          (1, 4), (1, 5))})

In [ ]:
print(f"Best score: {round(grid_search.best_score_, 3)}\n")

print("Best parameters: ")
params = grid_search.best_estimator_.get_params()
for param_name in parameters.keys():
    print(f"{param_name} = {params[param_name]}")

Best score: 0.984

Best parameters: 
countvectorizer__max_features = None
countvectorizer__ngram_range = (1, 2)


In [ ]:
ngram = make_pipeline(
    CountVectorizer(max_features = None, ngram_range = (1, 2), stop_words = noise),
    MultinomialNB()
)

ngram.fit(X_train, y_train)

ngram_pred = ngram.predict(X_test)

In [ ]:
pipeline = make_pipeline(
    TfidfVectorizer(stop_words = noise),
    MultinomialNB()
)

parameters = {
    'tfidfvectorizer__max_df': (0.75, 0.9),
    'tfidfvectorizer__min_df': (0.05, 0.1, 0.2),
    'tfidfvectorizer__max_features': (None, 500, 1000, 10000),
    'tfidfvectorizer__ngram_range': ((1, 2), (1, 3)),
}

grid_search = GridSearchCV(pipeline, parameters)

import warnings
warnings.filterwarnings("ignore")

grid_search.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('tfidfvectorizer',
                                        TfidfVectorizer(stop_words=['i', 'me',
                                                                    'my',
                                                                    'myself',
                                                                    'we', 'our',
                                                                    'ours',
                                                                    'ourselves',
                                                                    'you',
                                                                    "you're",
                                                                    "you've",
                                                                    "you'll",
                                                                    "you'd",
                                                                    'your',
                                                                    'yours',
                                                                    'yourself',
                                                                    'yourselves',
                                                                    'he', 'him',
                                                                    'his',
                                                                    'himself',
                                                                    'she',
                                                                    "she's",
                                                                    'her',
                                                                    'hers',
                                                                    'herself',
                                                                    'it',
                                                                    "it's",
                                                                    'its',
                                                                    'itself', ...])),
                                       ('multinomialnb', MultinomialNB())]),
             param_grid={'tfidfvectorizer__max_df': (0.75, 0.9),
                         'tfidfvectorizer__max_features': (None, 500, 1000,
                                                           10000),
                         'tfidfvectorizer__min_df': (0.05, 0.1, 0.2),
                         'tfidfvectorizer__ngram_range': ((1, 2), (1, 3))})

In [ ]:
print(f"Best score: {round(grid_search.best_score_, 3)}\n")

print("Best parameters: ")
params = grid_search.best_estimator_.get_params()
for param_name in parameters.keys():
    print(f"{param_name} = {params[param_name]}")

Best score: 0.867

Best parameters: 
tfidfvectorizer__max_df = 0.75
tfidfvectorizer__min_df = 0.05
tfidfvectorizer__max_features = None
tfidfvectorizer__ngram_range = (1, 2)


In [ ]:
tf_idf = make_pipeline(
    TfidfVectorizer(max_df = 0.75, min_df = 0, max_features = 500, ngram_range = (1, 2), stop_words = noise),
    MultinomialNB()
)

tf_idf.fit(X_train, y_train)

tf_idf_pred = tf_idf.predict(X_test)

In [ ]:
pipeline = make_pipeline(
    CountVectorizer(analyzer = 'char'),
    MultinomialNB()
)

parameters = {
    'countvectorizer__ngram_range': ((2, 4), (3, 6), (2, 8), (3, 8)),
}

grid_search = GridSearchCV(pipeline, parameters)

grid_search.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('countvectorizer',
                                        CountVectorizer(analyzer='char')),
                                       ('multinomialnb', MultinomialNB())]),
             param_grid={'countvectorizer__ngram_range': ((2, 4), (3, 6),
                                                          (2, 8), (3, 8))})

In [ ]:
print(f"Best score: {round(grid_search.best_score_, 3)}\n")

print("Best parameters: ")
params = grid_search.best_estimator_.get_params()
for param_name in parameters.keys():
    print(f"{param_name} = {params[param_name]}")

Best score: 0.987

Best parameters: 
countvectorizer__ngram_range = (2, 4)


In [ ]:
char_ngram = make_pipeline(
    CountVectorizer(max_features = None, ngram_range = (1, 4), analyzer = 'char'),
    MultinomialNB()
)

char_ngram.fit(X_train, y_train)

char_ngram_pred = char_ngram.predict(X_test)

In [ ]:
results = {
    'precision': [round(precision_score(y_test.values, ngram_pred, pos_label="spam"), 3), round(precision_score(y_test, tf_idf_pred, pos_label="spam"), 3), round(precision_score(y_test, char_ngram_pred, pos_label="spam"), 3)],
    'recall': [round(recall_score(y_test, ngram_pred, pos_label="spam"), 3), round(recall_score(y_test, tf_idf_pred, pos_label="spam"), 3), round(recall_score(y_test, char_ngram_pred, pos_label="spam"), 3)],
    'f1-score': [round(f1_score(y_test, ngram_pred, pos_label="spam"), 3), round(f1_score(y_test, tf_idf_pred, pos_label="spam"), 3), round(f1_score(y_test, char_ngram_pred, pos_label="spam"), 3)],
    'accuracy': [round(accuracy_score(y_test, ngram_pred), 3), round(accuracy_score(y_test, tf_idf_pred), 3), round(accuracy_score(y_test, char_ngram_pred), 3)]
}

pd.DataFrame(data = results, index = [['n-gram', 'tf-idf', 'Char n-gram']])

,precision,recall,f1-score,accuracy
n-gram,0.981,0.921,0.950,0.987
tf-idf,0.960,0.846,0.900,0.974
Char n-gram,0.972,0.912,0.941,0.984


## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах.

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **а?** - ноль или один символ **а**
* **а+** - один или более символов **а**
* **а\*** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа


In [ ]:
result = re.findall('a?b.', 'aabbсabbcbb')
print(result)

['abb', 'abb', 'bb']


In [ ]:
result = re.findall('a*b.', 'aabbсabbcbb')
print(result)

['aabb', 'abb', 'bb']


In [ ]:
result = re.findall('a+b.', 'aabbсabbcbb')
print(result)

['aabb', 'abb']


Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**:
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc')
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?

Потому что идет поиск непересекающихся совпадений

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
result = re.findall(r'\b\w{1,2}', 'abcd ef d ghijkab cabcxabc')
print(result)

['ab', 'ef', 'd', 'gh', 'ca']


### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie')
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2)
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
result = re.split('[.]', 'i fd s ftsy. b dsgsfitsy.  dsgsg sdg teenie. asd asd asdweenie.', maxsplit=2)
print(result)

['i fd s ftsy', ' b dsgsfitsy', '  dsgsg sdg teenie. asd asd asdweenie.']


### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
result = re.sub('\d', 'DIG', '123 asfj;k aj fj 2')
print (result)

DIGDIGDIG asfj;k aj fj DIG


**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
result = re.sub(r'http[s]?://([^ ])+\.[a-zA-Z]{2,4}/([^ ])*', '', 'Регулярные выражения в Python. Источник: https://habr.com/ru/articles/349860/')
print(result)

Регулярные выражения в Python. Источник: 


### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
prog = re.compile(r'\w{4,}')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'больше', 'больше', 'слов']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
text = 'abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz, asd@asdsd'
prog = re.compile(r'@\w+\.[a-zA-Z]{2,4}')
prog.findall(text)

['@gmail.com', '@test.in', '@analyticsvidhya.com', '@rest.biz']